# Foursquare and Yelp EDA
This notebook contains code for acquiring data from the Foursquare and Yelp APIs. 

This notebook is structured as follows: 

0. Prep Work for API Data Acquisition
1. Foursquare Data Acquisition from API, Wrangling, Data Audit, and Cleaning
2. Yelp Data Acquisition from API, Wrangling, Data Audit, and Cleaning
3. Results Comparison from Foursquare and Yelp Data

The following topics from the evaluation rubric are covered in this notebook:

- Data Acquisition
- API Interaction
- Data Wrangling
- Data Cleaning
- Data Audit

# 0. Prep Work for API Data Acquisition

In [57]:
# Import required packages
import requests
from pprint import pprint
import pandas as pd
import json
from IPython.display import JSON
import numpy as np
import glob

In [2]:
#Load csv with bike station ids and lat/longs
path = "C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/df_ids_latlongs.csv"
df_ids_latlongs = pd.read_csv(path)
df_ids_latlongs.head(1)

,Station ID,latitude,longitude
0,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397


# 1. Foursquare Data Acquisition from API, Wrangling, Data Audit, and Cleaning

<b>Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. </b>

<span style="color: red;">Note: Due to the fact that the Yelp and Foursquare APIs can only return 50 results per request, <b>I reduced the radius to 100 m</b> so that there would be variation in the number of businesses near bike stations.</span>

In [3]:
#Create inputs for Foursquare GET request
fs_url = 'https://api.foursquare.com/v3/places/search'
fs_key = open(r"C:\Users\mnicn\local_documents\lighthouse-data-notes\Week 3\fs.txt").readlines()[0]
fs_headers = {
    "accept": "application/json",
    "Authorization": fs_key
}

In [4]:
#Set variables for each category we will be examining
bars = '13003' #FS category: Dining and Drinking > Bar
restaurants = '13065' #FS category: Dining and Drinking > Restaurant

In [5]:
#Set fixed parameters for Foursquare GET request
fs_radius = 100 #search smaller radius to get some variability in the number of businesses returned
fs_limit = 50 #max number of returned results (50 is the max possible for the API)
fs_fields = 'fsq_id,name,rating,stats,popularity,price'
fs_categories = bars + "," + restaurants
print(fs_categories) #to ensure string formatted correctly

13003,13065


In [6]:
#Define function for get requests
def fs_request(lat_long):
    fs_params = {
    'll':lat_long,
    'radius':fs_radius,
    'limit':fs_limit,
    'fields':fs_fields,
    'categories':fs_categories
    }
    fs_response = requests.get(fs_url, headers=fs_headers, params=fs_params)
    resulting_json = json.loads(fs_response.text)
    return resulting_json

<b>Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)</b>

<b>Put your parsed results into a DataFrame</b>

In [7]:
#Define function to transform the json file into a dataframe. 

def json_to_df(json,station_id,i):
    resulting_df = pd.json_normalize(json['results'])

    #Add station id to df
    resulting_df['station_id'] = station_id

    #Save as csv
    path = "C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/Foursquare JSONs/fs_request" + "_" + str(i) + ".csv"
    resulting_df.to_csv(path, index=False)

In [8]:
#Create loop which incorporates two functions above and saves the resulting dfs into 1 csv each.
i = 0

while i < df_ids_latlongs.shape[0]:
    station_id = df_ids_latlongs.iloc[i,0]
    lat_long = str(df_ids_latlongs.iloc[i,1]) + "," + str(df_ids_latlongs.iloc[i,2])
    
    #Make fs_request using predefined function
    resulting_json = fs_request(lat_long)
    
    #Copy to csv if not a blank list 
    if resulting_json['results'] != []:
   
    #Using definted function, convert json to df and save resulting JSON to file. Files will be numbered 0, 1, 2,...
        json_to_df(resulting_json, station_id, i)
    
    i += 1

In [9]:
# Read all csv files from the following folder
files = glob.glob('C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/Foursquare JSONs/*.csv')

# Create an empty list to store data frames
frame = []

for file in files:
    frame.append(pd.read_csv(file))

# Concatenate Dataframes into a single Dataframe
fs_complete_df = pd.concat(frame, ignore_index=True)
fs_complete_df

,fsq_id,name,popularity,price,rating,stats.total_photos,stats.total_ratings,stats.total_tips,station_id
0,4fd3a01be4b0540d9cb027ea,Moii Cafe,0.221702,1.0,8.0,67.0,55.0,18.0,7a19c49f486d7c0c02b3685d7b240448
1,5d2843a1c56f480023f8e4cd,Jugo Juice,0.520996,1.0,6.4,12.0,8.0,1.0,7a19c49f486d7c0c02b3685d7b240448
2,506241a2e4b061897f154545,Pho Extreme Xe Lua,0.893123,2.0,5.6,44.0,57.0,18.0,7a19c49f486d7c0c02b3685d7b240448
3,5678df11498e17b35a76bc68,Donair Spot,0.212064,1.0,NaN,NaN,NaN,NaN,7a19c49f486d7c0c02b3685d7b240448
4,acd8424b3fad4dda1e267983,Mexicano Taco Ltd,NaN,NaN,NaN,NaN,NaN,NaN,7a19c49f486d7c0c02b3685d7b240448
...,...,...,...,...,...,...,...,...,...
1214,4d320c5498336dcb18401ff0,Cactus Club Cafe,0.996114,3.0,8.7,721.0,627.0,100.0,f3708906ec71348f5a42579fede36a0d
1215,5baeb74073fe25002c5728d5,LOT185 Cafe + Wine Bar,0.972329,1.0,NaN,NaN,NaN,NaN,f3708906ec71348f5a42579fede36a0d
1216,b950465a81844c7d30ada9e8,Feastro,NaN,NaN,NaN,NaN,NaN,NaN,f3708906ec71348f5a42579fede36a0d
1217,a09c441d051b4de1c05dacc3,Cactus Club Cafe Coal Harbour,NaN,NaN,NaN,NaN,NaN,NaN,f3708906ec71348f5a42579fede36a0d


In [10]:
#copy to csv
fs_complete_df.to_csv("C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/all_raw_fs_data.csv", index = False)

In [11]:
#Create intermediary table to relate bike stations and businesses
fsq_id_station_ids = fs_complete_df[['fsq_id','station_id']]

#copy to csv
fsq_id_station_ids.to_csv("C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/fsq_id_station_ids.csv", index = False)

fsq_id_station_ids.head()

,fsq_id,station_id
0,4fd3a01be4b0540d9cb027ea,7a19c49f486d7c0c02b3685d7b240448
1,5d2843a1c56f480023f8e4cd,7a19c49f486d7c0c02b3685d7b240448
2,506241a2e4b061897f154545,7a19c49f486d7c0c02b3685d7b240448
3,5678df11498e17b35a76bc68,7a19c49f486d7c0c02b3685d7b240448
4,acd8424b3fad4dda1e267983,7a19c49f486d7c0c02b3685d7b240448


## Data Cleaning and Data Audit

In [79]:
#Drop station id columns (the relationship is stored in an intermediary table)
fs_biz_info_df = fs_complete_df[['fsq_id','name','popularity','price','rating','stats.total_photos','stats.total_ratings','stats.total_tips']]
fs_biz_info_df.head(2)

#Drop all duplicates of fsq_id
#This table is to be a unique table of businesses, relationships are stored in an intermediary table. 
fs_biz_info_df = fs_biz_info_df.drop_duplicates(subset='fsq_id', keep='first', ignore_index=True)
fs_biz_info_df.head(2)

,fsq_id,name,popularity,price,rating,stats.total_photos,stats.total_ratings,stats.total_tips
0,4fd3a01be4b0540d9cb027ea,Moii Cafe,0.221702,1.0,8.0,67.0,55.0,18.0
1,5d2843a1c56f480023f8e4cd,Jugo Juice,0.520996,1.0,6.4,12.0,8.0,1.0


In [80]:
#Ensure that fsq_id columns only have unique values
fs_biz_info_df['fsq_id'].is_unique

True

In [81]:
fs_biz_info_df

,fsq_id,name,popularity,price,rating,stats.total_photos,stats.total_ratings,stats.total_tips
0,4fd3a01be4b0540d9cb027ea,Moii Cafe,0.221702,1.0,8.0,67.0,55.0,18.0
1,5d2843a1c56f480023f8e4cd,Jugo Juice,0.520996,1.0,6.4,12.0,8.0,1.0
2,506241a2e4b061897f154545,Pho Extreme Xe Lua,0.893123,2.0,5.6,44.0,57.0,18.0
3,5678df11498e17b35a76bc68,Donair Spot,0.212064,1.0,NaN,NaN,NaN,NaN
4,acd8424b3fad4dda1e267983,Mexicano Taco Ltd,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1149,4d320c5498336dcb18401ff0,Cactus Club Cafe,0.996114,3.0,8.7,721.0,627.0,100.0
1150,5baeb74073fe25002c5728d5,LOT185 Cafe + Wine Bar,0.972329,1.0,NaN,NaN,NaN,NaN
1151,b950465a81844c7d30ada9e8,Feastro,NaN,NaN,NaN,NaN,NaN,NaN
1152,a09c441d051b4de1c05dacc3,Cactus Club Cafe Coal Harbour,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
#Set fsq_id as the index
fs_biz_info_df = fs_biz_info_df.set_index('fsq_id')

In [90]:
#Rename columns for clarity
fs_col_names = ['name','popularity','price','rating','total_photos','total_ratings','total_tips']
fs_biz_info_df.columns = fs_col_names

In [91]:
fs_biz_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1154 entries, 4fd3a01be4b0540d9cb027ea to 4c0b27b970f241821848edbb
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           1154 non-null   object 
 1   popularity     867 non-null    float64
 2   price          791 non-null    float64
 3   rating         542 non-null    float64
 4   total_photos   537 non-null    float64
 5   total_ratings  542 non-null    float64
 6   total_tips     537 non-null    float64
dtypes: float64(6), object(1)
memory usage: 72.1+ KB


In [92]:
fs_biz_info_df.describe()

,popularity,price,rating,total_photos,total_ratings,total_tips
count,867.000000,791.000000,542.000000,537.000000,542.000000,537.000000
mean,0.841119,1.654867,7.268819,86.638734,70.206642,20.510242
std,0.209425,0.658709,0.966369,133.836514,99.432111,43.074826
min,0.000855,1.000000,4.900000,0.000000,0.000000,0.000000
25%,0.832932,1.000000,6.600000,16.000000,17.000000,5.000000
50%,0.917569,2.000000,7.300000,42.000000,38.000000,11.000000
75%,0.956618,2.000000,8.000000,95.000000,80.000000,22.000000
max,0.998756,4.000000,9.300000,980.000000,1123.000000,831.000000


In [96]:
#Save to csv
fs_biz_info_df.to_csv("C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/fs_businesses_unique_clean.csv")

# 2. Yelp Data Acquisition from API, Wrangling, Data Audit, and Cleaning

<b>Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. </b>

<span style="color: red;">Note: Due to the fact that the Yelp and Foursquare APIs can only return 50 results per request, <b>I reduced the radius to 100 m</b> so that there would be variation in the number of businesses near bike stations.</span>

In [13]:
#Create inputs for Foursquare GET request
y_url = 'https://api.yelp.com/v3/businesses/search'
y_key = open(r'C:\Users\mnicn\local_documents\lighthouse-data-notes\Week 3\Yelp2.txt').readlines()[0]
y_headers = {
    'Authorization': 'Bearer %s' % y_key
}

In [14]:
#Set variables for each category we will be examining
y_bars = 'bars'
y_restaurants = 'restaurants'

In [15]:
#Set fixed parameters for Yelp GET request
y_radius = 100 #search smaller radius to get some variability in the number of businesses returned
y_limit = 50 #max number of returned results (50 is the max possible for the API)
y_categories = y_bars + "," + y_restaurants
print(y_categories) #to ensure string formatted correctly

bars,restaurants


In [16]:
test_lat = 49.262487
test_long = -123.114397

In [17]:
#Define function for get requests
def y_request(lat,long):
    y_params = {
        'latitude':lat,
        'longitude':long,
        'radius':y_radius,
        'limit':y_limit,
        'categories':'y_categories'
    }
    y_response = requests.get(y_url, headers=y_headers, params=y_params)
    resulting_json = json.loads(y_response.text)
    return resulting_json

<b>Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)</b>

<b>Put your parsed results into a DataFrame</b>

In [18]:
#Define function to transform the json file into a dataframe. 

def y_json_to_df(json,station_id,i):    
        resulting_df = pd.json_normalize(json['businesses'])
        
        #Only keep relevant columns
        resulting_df = resulting_df[['id','name','review_count','rating','transactions']]

        #Add station id to df
        resulting_df['station_id'] = station_id

        #Save as csv
        path = "C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/Yelp JSONs/y_request" + "_" + str(i) + ".csv"
        resulting_df.to_csv(path, index=False)

In [19]:
#Create loop which incorporates two functions above and saves the resulting dfs into 1 csv each.
i = 0

while i < df_ids_latlongs.shape[0]:
    station_id = df_ids_latlongs.iloc[i,0]
    lat = df_ids_latlongs.iloc[i,1]
    long = df_ids_latlongs.iloc[i,2]
    
    #Make y_request using predefined function
    resulting_json = y_request(lat, long)
    
    #Copy to csv if not a blank list 
    if len(resulting_json['businesses']) != 0:
   
    #Using definted function, convert json to df and save resulting JSON to file. Files will be numbered 0, 1, 2,...
        y_json_to_df(resulting_json, station_id, i)
    
    i += 1

In [20]:
# Read all csv files from the following folder
files = glob.glob('C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/Yelp JSONs/*.csv')

# Create an empty list to store data frames
frame = []

for file in files:
    frame.append(pd.read_csv(file))

# Concatenate Dataframes into a single Dataframe
y_complete_df = pd.concat(frame, ignore_index=True)
y_complete_df

,id,name,review_count,rating,transactions,station_id
0,NensKn1MSVU_rm-1Y6WlFA,Marulilu Cafe,281,4.0,[],7a19c49f486d7c0c02b3685d7b240448
1,oHYOa9l5aLOQpJ8dF6sDmA,Menya Raizo,68,4.0,[],7a19c49f486d7c0c02b3685d7b240448
2,3sTVFA9wQQT_7RTUj_g1rw,Donair Spot,42,4.0,[],7a19c49f486d7c0c02b3685d7b240448
3,1AA4EjIIXywxOSnbyZ9vLA,Pho Extreme Xe Lua,101,2.5,[],7a19c49f486d7c0c02b3685d7b240448
4,31ag-M_QiaycGQrvAi169w,Jollibee,6,4.5,[],7a19c49f486d7c0c02b3685d7b240448
...,...,...,...,...,...,...
1595,kRfJn1Fwcl3iXmYhKalI5w,Jack Poole Plaza,4,5.0,[],f3708906ec71348f5a42579fede36a0d
1596,4f8mgwrPV_XRXMP9qLVhow,Starbucks,11,3.5,[],f3708906ec71348f5a42579fede36a0d
1597,e8eDD0Cn_cRNoFhj90H7sw,Caffè Super Veloce,3,4.5,[],f3708906ec71348f5a42579fede36a0d
1598,FwaxOvge4_iOmivWn20Zng,Nike - Greek God,3,4.0,[],f3708906ec71348f5a42579fede36a0d


In [21]:
#copy to csv
y_complete_df.to_csv("C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/all_raw_yelp_data.csv", index = False)

In [22]:
#Create intermediary table to relate bike stations and businesses
y_id_station_ids = y_complete_df[['id','station_id']]

#copy to csv
y_id_station_ids.to_csv("C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/y_id_station_ids.csv", index = False)

y_id_station_ids.head()

,id,station_id
0,NensKn1MSVU_rm-1Y6WlFA,7a19c49f486d7c0c02b3685d7b240448
1,oHYOa9l5aLOQpJ8dF6sDmA,7a19c49f486d7c0c02b3685d7b240448
2,3sTVFA9wQQT_7RTUj_g1rw,7a19c49f486d7c0c02b3685d7b240448
3,1AA4EjIIXywxOSnbyZ9vLA,7a19c49f486d7c0c02b3685d7b240448
4,31ag-M_QiaycGQrvAi169w,7a19c49f486d7c0c02b3685d7b240448


In [23]:
y_id_station_ids.head()

,id,station_id
0,NensKn1MSVU_rm-1Y6WlFA,7a19c49f486d7c0c02b3685d7b240448
1,oHYOa9l5aLOQpJ8dF6sDmA,7a19c49f486d7c0c02b3685d7b240448
2,3sTVFA9wQQT_7RTUj_g1rw,7a19c49f486d7c0c02b3685d7b240448
3,1AA4EjIIXywxOSnbyZ9vLA,7a19c49f486d7c0c02b3685d7b240448
4,31ag-M_QiaycGQrvAi169w,7a19c49f486d7c0c02b3685d7b240448


In [130]:
y_complete_df

,id,name,review_count,rating,transactions,station_id
0,NensKn1MSVU_rm-1Y6WlFA,Marulilu Cafe,281,4.0,[],7a19c49f486d7c0c02b3685d7b240448
1,oHYOa9l5aLOQpJ8dF6sDmA,Menya Raizo,68,4.0,[],7a19c49f486d7c0c02b3685d7b240448
2,3sTVFA9wQQT_7RTUj_g1rw,Donair Spot,42,4.0,[],7a19c49f486d7c0c02b3685d7b240448
3,1AA4EjIIXywxOSnbyZ9vLA,Pho Extreme Xe Lua,101,2.5,[],7a19c49f486d7c0c02b3685d7b240448
4,31ag-M_QiaycGQrvAi169w,Jollibee,6,4.5,[],7a19c49f486d7c0c02b3685d7b240448
...,...,...,...,...,...,...
1595,kRfJn1Fwcl3iXmYhKalI5w,Jack Poole Plaza,4,5.0,[],f3708906ec71348f5a42579fede36a0d
1596,4f8mgwrPV_XRXMP9qLVhow,Starbucks,11,3.5,[],f3708906ec71348f5a42579fede36a0d
1597,e8eDD0Cn_cRNoFhj90H7sw,Caffè Super Veloce,3,4.5,[],f3708906ec71348f5a42579fede36a0d
1598,FwaxOvge4_iOmivWn20Zng,Nike - Greek God,3,4.0,[],f3708906ec71348f5a42579fede36a0d


## Data Cleaning and Data Audit

In [131]:
#Drop station id columns (the relationship is stored in an intermediary table)
y_biz_info_df = y_complete_df[['id','name','review_count','rating','transactions']]
y_biz_info_df.head(2)

#Drop all duplicates of id
y_biz_info_df = y_biz_info_df.drop_duplicates(subset='id', keep='first', ignore_index=True)
y_biz_info_df

,id,name,review_count,rating,transactions
0,NensKn1MSVU_rm-1Y6WlFA,Marulilu Cafe,281,4.0,[]
1,oHYOa9l5aLOQpJ8dF6sDmA,Menya Raizo,68,4.0,[]
2,3sTVFA9wQQT_7RTUj_g1rw,Donair Spot,42,4.0,[]
3,1AA4EjIIXywxOSnbyZ9vLA,Pho Extreme Xe Lua,101,2.5,[]
4,31ag-M_QiaycGQrvAi169w,Jollibee,6,4.5,[]
...,...,...,...,...,...
1429,kRfJn1Fwcl3iXmYhKalI5w,Jack Poole Plaza,4,5.0,[]
1430,4f8mgwrPV_XRXMP9qLVhow,Starbucks,11,3.5,[]
1431,e8eDD0Cn_cRNoFhj90H7sw,Caffè Super Veloce,3,4.5,[]
1432,FwaxOvge4_iOmivWn20Zng,Nike - Greek God,3,4.0,[]


In [132]:
#Ensure that id columns only have unique values
y_biz_info_df['id'].is_unique

True

In [133]:
#Set y_id as the index
y_biz_info_df = y_biz_info_df.set_index('id')

In [134]:
y_biz_info_df.head(2)

,name,review_count,rating,transactions
id,,,,
NensKn1MSVU_rm-1Y6WlFA,Marulilu Cafe,281,4.0,[]
oHYOa9l5aLOQpJ8dF6sDmA,Menya Raizo,68,4.0,[]


In [135]:
#Rename columns for clarity
y_col_names = ['name','review_count','rating','number_transactions']
y_biz_info_df.columns = y_col_names

In [136]:
y_biz_info_df.head(2)

,name,review_count,rating,number_transactions
id,,,,
NensKn1MSVU_rm-1Y6WlFA,Marulilu Cafe,281,4.0,[]
oHYOa9l5aLOQpJ8dF6sDmA,Menya Raizo,68,4.0,[]


In [137]:
y_biz_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1434 entries, NensKn1MSVU_rm-1Y6WlFA to cALV8k0kRonC_VD2yMzAXA
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 1434 non-null   object 
 1   review_count         1434 non-null   int64  
 2   rating               1434 non-null   float64
 3   number_transactions  1434 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 56.0+ KB


In [138]:
y_biz_info_df.describe()

,review_count,rating
count,1434.000000,1434.000000
mean,75.109484,3.677824
std,150.827496,0.816634
min,1.000000,1.000000
25%,6.000000,3.500000
50%,24.500000,4.000000
75%,84.000000,4.000000
max,2505.000000,5.000000


In [139]:
y_biz_info_df["number_transactions"].value_counts()

[]                            1430
['restaurant_reservation']       4
Name: number_transactions, dtype: int64

In [140]:
#'number_transactions' column does not appear to contain useful information; drop
y_biz_info_df = y_biz_info_df.drop(labels="number_transactions", axis=1)
y_biz_info_df.head()

,name,review_count,rating
id,,,
NensKn1MSVU_rm-1Y6WlFA,Marulilu Cafe,281,4.0
oHYOa9l5aLOQpJ8dF6sDmA,Menya Raizo,68,4.0
3sTVFA9wQQT_7RTUj_g1rw,Donair Spot,42,4.0
1AA4EjIIXywxOSnbyZ9vLA,Pho Extreme Xe Lua,101,2.5
31ag-M_QiaycGQrvAi169w,Jollibee,6,4.5


In [141]:
#Save to csv
y_biz_info_df.to_csv("C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/y_businesses_unique_clean.csv")

# 3. Results Comparison from Foursquare and Yelp Data

<b>Which API provided you with more complete data? Provide an explanation. </b>

|Attribute|Foursquare|Yelp|
|---|---|---|
|Unique businesses found within 100 m of bike stations|1,154|1,433|
|Attributes Available|Foursquare was able to provide more information about businesses, including statistics such as photos, number of ratings, and number of tips|Yelp was able to provide basic information. Price information was available but I was running into an error when I tried to gather price data, so I did not collect this statistic|
|Data quality - nulls|There were many nulls in the Foursquare data, especially for the detailed business information. Basic business information, such as popularity and price, were approximately 75% non-null.|No nulls were found in the basic business information|
|Scales of Data|Foursquare are from 1-10, and their pricing info is rated as 1, 2, 3, or 4|Yelp ratings are from 1-4|
|Number of Review|Foursquare businesses have an average of 70 ratings.|Yelp businesses have an average of 75 ratings. 

<b><span style="color:red">Due to the larger number of businesses found by Yelp and the completeness of data (no nulls), I will do the following steps of the project using Yelp data</b></span>

<b>Get the top 10 restaurants according to their rating</b>

Notes: 
- I pulled all bars and restaurants from the Foursquare and Yelp APIs, but I didn't pull data on the specific categories (ie. whether a business was a bar or a restaurant). It appears that some of the places that were returned are not *primarily* bars or restaurants, such as FlyOver Canada and Stanley Park. If I had more time to do this project, I may only be interested in restaurants which were *primarily* restaurants or bars, and would pull more category data from the API to determine this and clean accordingly. 
- I googled a couple of the restaurants that appeared as top 10 restaurants and found that data (at least in the Yelp dataset) includes businesses which are *permanently closed*. If I had more time to do this project, I would pull more data from the API (if possible) to determine whether restaurants were open and clean accordingly.
- The businesses with the top 10 ratings from Foursquare all have many ratings (38 or more), while many of the businesses with the top 10 ratings from Yelp have only 1 review. Based on this, it seems likely that Foursquare's rating is not a straight average of the ratings from customer reviews, but based on an internal calculation that takes into account number of reviews, average of reviews, and possibly other factors. On the other hand, Yelp's 'rating' appears to be a straight average of customer reviews. 
   - Based on this information, as a customer, I would trust Foursquare's data more, since a single-five start review on Yelp would give a restaurant five stars.

In [142]:
#Read from csvs
fs_businesses_unique_clean = pd.read_csv("C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/fs_businesses_unique_clean.csv")
y_businesses_unique_clean = pd.read_csv("C:/Users/mnicn/local_documents/lighthouse-data-notes/Week 3/Project 2 Fresh Files/data/y_businesses_unique_clean.csv")

In [152]:
#Sort on ratings
fs_businesses_unique_clean.sort_values("rating", ascending=False).head(10)

,fsq_id,name,popularity,price,rating,total_photos,total_ratings,total_tips
367,54bea64f498ec3ed3dde9a10,Earnest Ice Cream,0.984707,1.0,9.3,253.0,300.0,51.0
1042,5075f7b9f2e73b925f6431ef,FlyOver Canada,0.998756,NaN,9.3,222.0,369.0,41.0
953,51b201d27dd249ae714ba728,33 Acres Brewing Co,0.959464,2.0,9.2,591.0,388.0,75.0
462,5646ab13498e460c732920e8,Ramen Danbo - Robson,0.989682,2.0,9.1,224.0,197.0,43.0
1067,55c40e66498e739ed2c87adf,The Mackenzie Room,0.986320,2.0,9.1,26.0,38.0,7.0
944,4d2cce46ae3a8cfa4067bf70,Hawksworth Restaurant,0.985581,4.0,9.1,401.0,270.0,66.0
1056,5780683c498e51523ab4851b,Raisu,0.980548,2.0,9.1,143.0,67.0,10.0
1051,50d4bd11e4b0b55bb4b657a3,Beaucoup Bakery & Cafe,0.969472,2.0,9.0,377.0,287.0,95.0
1040,4aa7f561f964a520384e20e3,Miku,0.996055,3.0,9.0,959.0,610.0,178.0
398,4aa7455df964a5208e4c20e3,Kingyo Izakaya 金魚居酒屋,0.976856,3.0,9.0,766.0,373.0,100.0


In [149]:
y_businesses_unique_clean.sort_values("rating", ascending=False).head(10)

,id,name,review_count,rating
1027,gP6UmMKWA-YOVRiDVHTZNA,1 up caramels,1,5.0
462,QMHwiOLZgoEiUHZcSgJuJw,Taxi Café,1,5.0
798,HJNwJpfspIhfvqtr4cYE2g,Caffe Mira,15,5.0
338,zu58Fz_lur97NGf4GjwQYA,Word.,1,5.0
1263,RNgR9Y_byajHqgVxP0dnKA,Cardero Cafe,14,5.0
426,i3H6ZE5Qokd8exdKslTLpQ,Valendine,1,5.0
795,kDZTaeUaf78Jd25ktX8a8g,Number e food,134,5.0
1270,720w4KP2M1-xpTRAc2QoPQ,Gringo,1,5.0
1168,OXEaIQsgJIUCyRev5VgvLA,NUTTEA,19,5.0
102,kajMc2fkWKdzKJ1M4pm47Q,Stanley Park,1034,5.0
